[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aprendizaje-automatico-dc-uba-ar/material/blob/main/notebooks/notebook_05_seleccion_modelos-published.ipynb)

# Selección de modelos

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display


## Cross validation 

Hasta ahora sólo habíamos visto (ver en el [notebook 03](https://github.com/aprendizaje-automatico-dc-uba-ar/material/blob/main/notebooks/notebook_03_arboles_de_decision_sklearn-published.ipynb)) que ibamos a dividir los datos en train y test.


En esta semana vimos la opción de hacer validación cruzada. En esta oportunidad lo que haremos será realizar una exploración de hiperparámetros para árboles incorporando conceptos de la clase de esta semana.
Vamos a experimentar usando k-fold (con k=10) para explorar distintos valores de configuración de `DecisionTreeClassifier` para seleccionar el hiperparámetro que nos parezca el mejor. 
Ensayaremos áltura máxima con valores `[None, 1, 2, 3, 5, 8, 13, 21]`. 

Nos interesará:
- controlar el tiempo de entrenamiento
- generar alguna métrica que elijamos para seleccionar la áltura máxima

Con la mejor configuración obtenida entrenar un clasificador con todos los datos de desarrollo.
    
Evaluar el comportamiento con el set de evaluación
    



In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier
import timeit
import pandas as pd

def cargar_datos():
    df = pd.read_csv('https://github.com/aprendizaje-automatico-dc-uba-ar/material/raw/main/datasets/data_05/seleccion_modelos.csv')
    X = df.drop("Y", axis=1)
    y = df.Y
    return X.to_numpy(), y.to_numpy()

X, y = cargar_datos()
X,y

Primero separaremos nuestro data set entre **desarrollo** y **evaluación** en un 10%. Para esto podemos usar `train_test_split`

In [ ]:
# separamos entre dev y eval
X_dev, X_eval, y_dev, y_eval = train_test_split(
                    X, 
                    y, 
                    random_state=4, 
                    test_size=0.1)
type(y_eval)

Por el momento dejaremos el set de evaluación de lado y nos manejaremos con el de desarrollo.

Pasemos a experimentar los distinos `h_max` posibles.

Usaremos estas dos funciones para entrenar un árbol y para usarlo para predecir respectivamente:

In [ ]:
def train_tree(X_tr: np.ndarray, y_tr: np.ndarray, tree_params={}) -> DecisionTreeClassifier:
    arbol = DecisionTreeClassifier(**tree_params)
    arbol.fit(X_tr, y_tr)

    return arbol

def tree_predict(ab: DecisionTreeClassifier, X_test: np.ndarray) -> np.ndarray:
    predictions = ab.predict(X_test)
    return predictions

Y definimos la métrica a usar. A modo de ejemplo figura accuracy.

Cambiar la medida por una nueva vista en clase.

In [ ]:
def accuracy(y_predicted: np.ndarray, y_real: np.ndarray) -> float:
    TP_TN = sum([y_i == y_j for (y_i, y_j) in zip(y_predicted, y_real)]) 
    P_N = len(y_real)
    return TP_TN /P_N

def metrica_seleccionada(y_predicted: np.ndarray, y_real: np.ndarray) -> float:
    return accuracy(y_predicted, y_real)

Realización del experimento.

Nota: se inicializa con una semilla para poder reproducir el resultado.

In [ ]:
results = []

np.random.seed(44)
for h_max in [None, 1, 2, 3, 5, 8, 13, 21]:
    kf = KFold(n_splits=10)    
    y_pred = np.empty(y_dev.shape)
    y_pred.fill(np.nan)
    
    # generamos para cada fold una predicción
    for train_index, test_index in kf.split(X_dev):
        
        #saco el fold que no uso para entrenar
        kf_X_train, kf_X_test = X_dev[train_index], X_dev[test_index]
        kf_y_train, kf_y_test = y_dev[train_index], y_dev[test_index]

        current_tree = train_tree(kf_X_train, kf_y_train,
                                    tree_params={"max_depth":h_max})
        predictions = tree_predict(current_tree, kf_X_test)
        y_pred[test_index] = predictions
        
    current_score = metrica_seleccionada(y_pred, y_dev)
        
    results.append((h_max,current_score))
    

# Ordenamos los resultados (puede ser que convenga del derecho o del reves)
r = sorted(results, key=lambda x: x[1], reverse=True)

print("Órden obtenido según la métrica elegida")
for idx, (h, sc) in enumerate(r):
    print(f"\t{idx+1}- h_max={h} con {sc:.3f}")


Con los resultados obtenidos podemos elegir la `h_max` que nos parezca mejor. Con eso vamos a reentrenar el modelo con todos los datos. 

¿Qué teníamos que tener en cuenta en estos casos?

In [ ]:
# elegimos
h_max = None # COMPLETAR
selection_score = None # COMPLETAR


In [ ]:
assert selection_score is not None, "Completar la celda anterior para continuar"

Con el mejor parámetro entrenamos un nuevo clasificador:

In [ ]:
print(f"Construimos nuestro clasificador con parámetro 'max_depth'={h_max}. "
     + f"Para seleccionarlo el score que habíamos obtenido era {selection_score:.3f}")

best_tree = train_tree(X_dev, y_dev,
                            tree_params={"max_depth":h_max})


Podemos evaluar este árbol en train.

In [ ]:
y_pred = tree_predict(best_tree, X_dev)       
best_tree_score = metrica_seleccionada(y_pred, y_dev)

¿Qué nos están diciendo estos dos scores?¿Para qué nos sirven?

Por única vez vemos como funciona nuestro entrenamiento en los datos de **evaluación** que no habíamos mirado.

In [ ]:
y_pred_eval = tree_predict(best_tree, X_eval)       
best_tree_score_eval = metrica_seleccionada(y_pred_eval, y_eval)

print(f"Con el árbol entrenado con el parámetro seleccionado tenemos en eval un score de {best_tree_score_eval:.3f}")

*Opcionales*

1. Simular qué hubiese ocurrido si hubieramos elegido un K distinto. ¿La diferencia entre el score en *dev* y el score en *eval* cambia significativamente?
2. Repetir el mismo ejercicio de elegir la mejor combinación de parametros pero esta vez establecer una grilla donde se exploren al menos dos hiperparámetros que no sean la altura máxima. Revisar la documentación de `DecisionTreeClassifier`, por ejemplo pueden elegir la **medida de impureza** y el **mínimo de muestas necesario para realizar un split**. Definir los rangos necesarios para explorar más de un valor de cada hiperparámetro considerado. ¿Este modelo fue mejor que el obtenido en el punto anterior?

**Importante**: en este punto nos tomamos la licencia de usar nuevamente el conjunto de evaluación. El re-uso del conjunto de evaluación sólo lo permitimos en este caso por motivos pedagócios. Pero **NO DEBE** suceder en la práctica.



## Group K-Fold


Cuando entrenamos modelos de machine learning, nuestro objetivo es evaluar correctamente su capacidad para generalizar a nuevos datos. Para ello, utilizamos técnicas de validación cruzada como K-Fold. Sin embargo, en algunos casos, el uso de K-Fold tradicional puede llevar a una sobreestimación del rendimiento del modelo si no se considera la estructura de los datos.

Por ejemplo, si queremos que nuestro modelo sea capaz de predecir emociones en personas que nunca ha visto, pero en los datos de entrenamiento y prueba aparecen instancias de las mismas personas, estaremos evaluando algo distinto a lo que realmente nos interesa. En este caso, deberíamos agrupar las instancias por el atributo `persona_id`.

Lo mismo ocurre en problemas médicos: si queremos que el modelo generalice a hospitales nuevos, pero en la validación se mezclan datos del mismo hospital entre el entrenamiento y la prueba, el modelo podría estar aprovechando características específicas de cada hospital en lugar de aprender patrones generales. Para evitarlo, debemos agrupar por `hospital_id`.

Utilizar `Group K-Fold` nos permite asegurarnos de que la validación refleje realmente la capacidad del modelo para generalizar, garantizando que los datos de un mismo grupo no aparezcan en ambos conjuntos (entrenamiento y validación). Así, obtenemos una estimación más realista de su desempeño en escenarios del mundo real.

In [ ]:
import numpy as np
import pandas as pd

# Simulación de datos médicos
np.random.seed(42)
n_samples = 1000
n_patients = 7

df = pd.DataFrame({
    'biomarker1': np.random.randn(n_samples),
    'biomarker2': np.random.randn(n_samples),
    'patient_id': np.random.randint(0, n_patients, n_samples),  # Cada paciente tiene múltiples muestras
    'disease': np.random.randint(0, 2, n_samples)  # 0 = sano, 1 = enfermo
})

Veamos cuánto es la estimación si usamos K-fold normal

In [ ]:
results_kfold = []

X = df[['biomarker1', 'biomarker2', 'patient_id']].values
y = df['disease'].values

np.random.seed(44)
for h_max in [None, 1, 2, 3, 5, 8, 13, 21]:
    kf = KFold(n_splits=5, shuffle=True)    
    y_pred = np.empty(y.shape)
    y_pred.fill(np.nan)

    # generamos para cada fold una predicción
    for train_index, test_index in kf.split(X):
        
        #saco el fold que no uso para entrenar
        kf_X_train, kf_X_test = X[train_index], X[test_index]
        kf_y_train, kf_y_test = y[train_index], y[test_index]

        current_tree = train_tree(kf_X_train, kf_y_train,
                                    tree_params={"max_depth":h_max})
        predictions = tree_predict(current_tree, kf_X_test)
        y_pred[test_index] = predictions
        
    current_score = metrica_seleccionada(y_pred, y)
        
    results_kfold.append((h_max,current_score))

In [ ]:
results_kfold

Ahora hagamos el mismo procedimiento pero agrupando por el atributo 'patient_id', considerando que vamos a tener un fold por paciente.

In [ ]:
results_group_kfold = []

X = df[['biomarker1', 'biomarker2']].values  # Características sin patient_id
y = df['disease'].values
groups = df['patient_id'].values  # Usamos patient_id como grupo

In [ ]:
# TODO: Implementar Group K-Fold manualmente
# 1. Obtener los grupos únicos
unique_groups = # COMPLETAR

# 2. Mezclar los grupos (para aleatorizar)
np.random.shuffle(unique_groups)

# 3. Dividir los grupos en folds (consideramos un grupo por fold)
n_splits = len(unique_groups)
group_folds = np.array_split(unique_groups, n_splits)

# 4. Para cada profundidad máxima del árbol que quieras evaluar:
for h_max in [None, 1, 2, 3, 5, 8, 13, 21]:
    # Inicializa el array para guardar las predicciones
    y_pred = np.empty(y.shape)
    y_pred.fill(np.nan)
    
    # Para cada fold:
    for i in range(n_splits):
        # TODO: Definir qué grupos van a test y cuáles a train
        # ...
        
        # TODO: Obtener los índices de las muestras para cada conjunto
        # ...
        
        # TODO: Separar los datos según los índices
        gkf_X_train, gkf_y_train =
        
        current_tree = train_tree(gkf_X_train, gkf_y_train,
                                  tree_params={"max_depth": h_max})
        predictions = tree_predict(current_tree, gkf_X_test)
        y_pred[test_indices] = predictions
    
    # Evaluamos el rendimiento global
    current_score = metrica_seleccionada(y, y_pred)
    results_group_kfold.append((h_max, current_score))

In [ ]:
results_group_kfold

Si utilizamos validación cruzada K-Fold tradicional para seleccionar y evaluar nuestro modelo en un conjunto de datos donde múltiples muestras provienen de los mismos pacientes, ¿qué ocurriría cuando intentemos predecir con ese modelo en datos de un paciente completamente nuevo? ¿Cuál sería la relación entre la performance estimada durante la validación y la performance real observada en la práctica clínica con pacientes nuevos?